In [1]:
import yaml
import torch
import lightning_toolbox as ltb
import lightning
import dypy as dy
from ocd.models.permutation.utils import all_permutations
import sys
import ocd
import random

# create a random permutation of n numbers

data_config = yaml.safe_load(
    open("experiments/intervention/data/parametric_laplace_modulated_25_1000_fork_sin_plus_x.yml", "r")
)
model_config = yaml.safe_load(open("experiments/intervention/model.yml", "r"))

n = data_config["init_args"]["dataset_args"]["scm_generator_args"]["graph_generator_args"]["n"]
model_config["init_args"]["model_args"]["in_features"] = n

# setup fixed perm
enforce_perm = torch.randperm(n).tolist()
data_config["init_args"]["dataset_args"]["scm_generator_args"]["graph_generator_args"][
    "enforce_ordering"
] = enforce_perm
model_config["init_args"]["model_args"]["ordering"] = enforce_perm
model_config["init_args"]["model_args"]['scale_transform_args'] = dict(post_act_scale=5.0, normalization=None)
trainer_config = yaml.safe_load(open("experiments/intervention/trainer.yml", "r"))
trainer_config["max_epochs"] = 400
trainer_config["enable_progress_bar"] = False
trainer_config["enable_model_summary"] = True
grad_clip_val = 1000

dm = ltb.DataModule(**data_config["init_args"])
model = dy.eval(model_config["class_path"])(**model_config["init_args"])

trainer = lightning.Trainer(
    **trainer_config,
    callbacks=[lightning.pytorch.callbacks.LearningRateMonitor()],
    gradient_clip_val=1.0,
    gradient_clip_algorithm='value',
)
model

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/vahidzee/.pyenv/versions/3.10.9/envs/deep/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


TrainingModule(
  (model): OCDAF(
    (flow): AffineFlow(
      (0): MaskedAffineFlowTransform(
        ordering=tensor([23,  5,  8,  3, 11, 18, 22, 16, 10,  7,  6, 17,  4,  1, 12, 13, 20,  9,
                19, 15,  0, 14, 21,  2, 24], dtype=torch.int32)
        (masked_mlp_shift): MaskedMLP(
          num_masks=1, seed=0
          (0): MaskedBlock(
            in_features=25, out_features=625, bias=True
            (activation): LeakyReLU(negative_slope=0.1)
          )
          (1): MaskedBlock(
            in_features=625, out_features=625, bias=True
            (activation): LeakyReLU(negative_slope=0.1)
          )
          (2): MaskedBlock(
            in_features=625, out_features=625, bias=True
            (activation): LeakyReLU(negative_slope=0.1)
          )
          (3): MaskedBlock(in_features=625, out_features=25, bias=True)
        )
        (masked_mlp_scale): MaskedMLP(
          num_masks=1, seed=0
          (0): MaskedBlock(
            in_features=25, out_featu

In [2]:
trainer.fit(model, dm)


  | Name  | Type  | Params
--------------------------------
0 | model | OCDAF | 1.6 M 
--------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.515     Total estimated model params size (MB)
/Users/vahidzee/.pyenv/versions/3.10.9/envs/deep/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/vahidzee/.pyenv/versions/3.10.9/envs/deep/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which i